In [2]:
%run Regard.py --no-cuda

cuda? False


AttributeError: 'torch.LongTensor' object has no attribute 'requires_grad'

In [4]:
%cat Regard.py

from __future__ import print_function
import argparse
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import matplotlib.pyplot as plt
import torchvision


# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=10, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=10, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, m

In [15]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import matplotlib.pyplot as plt
import torchvision


# Training settings
cuda = False
torch.manual_seed(42)

kwargs = {'num_workers': 1, 'pin_memory': True}

dataset = torchvision.datasets.ImageFolder('dataset',
                                        transforms.Compose([
                                        transforms.CenterCrop(100),
                                        transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                        ])) 
train_loader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=2)
test_loader= torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=2)

#classes = 'blink','left','right','center'

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)#
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(5000, args.dimension)# args.dimension prend la valeur default de l'argument dimension.
        self.fc2 = nn.Linear(args.dimension, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 4))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 4))
        x = x.view(-1, 5000)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))#
        x = F.dropout(x, training=self.training)
        x = self.fc3(x)#
        #x = self.fc2(x)
        return F.log_softmax(x, dim=1)#x

model = Net()
if args.cuda:
    model.cuda()

criterion = nn.CrossEntropyLoss()#
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):#,0):
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        print(data.size(), target.size())
        data, target = Variable(data), Variable(target) #[batch_idx].unsqueeze(0)
        print(data.size(), target.size())

        optimizer.zero_grad()
        output = model(data)
        #loss = F.nll_loss(output, target)
        loss = criterion(output, labels)#
        loss.backward()
        optimizer.step()
        if args.log_interval>0: # rajout de la commande pour pouvoir print ou non les différents epoch ou juste le résultat
            if batch_idx % args.log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.data[0]))

dataiter = iter(train_loader)
images, labels = dataiter.next()

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    if args.log_interval>0: print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return correct / len(test_loader.dataset)

def protocol():
    for epoch in range(10):
        train(epoch)
    return test()

def main():
    for epoch in range(10):
        train(epoch)
        Accuracy = test()
    print('Test set: Final Accuracy: {:.3f}%'.format(Accuracy*100)) # print que le pourcentage de réussite final
    
    
t0 = time.time () # ajout de la constante de temps t0

main()

t1 = time.time () # ajout de la constante de temps t1

print ("Le programme a mis",t1-t0, "secondes à s'exécuter.") #compare t1 et t0, connaitre le temps d'execution du programme

torch.Size([10, 3, 100, 100]) torch.Size([10])
torch.Size([10, 3, 100, 100]) torch.Size([10])


AttributeError: 'torch.LongTensor' object has no attribute 'requires_grad'